<a href="https://colab.research.google.com/github/alexmascension/ANMI/blob/main/notebook/T4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tema 5: Interpolación de funciones

In [ ]:
!pip install -r https://raw.githubusercontent.com/alexmascension/ANMI/main/requirements.txt

In [ ]:
from sympy import *
from sympy.matrices import Matrix as mat
from sympy.matrices import randMatrix
from sympy import symbols
import sympy

import numpy as np
from tqdm import tqdm

from scipy.linalg import orth

from matplotlib import pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 150

In [ ]:
from anmi.genericas import norma_p_func, norma_inf_func

In [ ]:
x, y, z, a, lambda_ = symbols('x'), symbols('y'), symbols('z'), symbols('a'), symbols('lambda')

Dado un conjunto de puntos (nodos) $x_0, x_1, \cdots, x_m$ podemos crear un polinomio $p$ que aproxime la función $f$ y que cumpla que $p(x_i) = f(x_i)$ para todo nodo. Existen varias situaciones de interpolación:
* El número de nodos menos 1, $m-1$ es MAYOR que el máximo grado de polinomios, $n$.
* El número de nodos menos 1, $m-1$ es IGUAL que el máximo grado de polinomios, $n$.
* El número de nodos menos 1, $m-1$ es MENOR que el máximo grado de polinomios, $n$.

En este capítulo nos centraremos cuando $m - 1 = n$.

## Interpolación de Lagrange

En esta interpolación empleamos un conjunto de funciones $L = \{l_0, l_1, \cdots, l_n\}$ correspondiente a los $n+1$ nodos $\{x_i:i = 0, 1, \cdots, n\}$. Cada $l_i$ se define como

$$l_i(x) = \prod_{k = 0 \neq i}\frac{x - x_k}{x_i - x_k}$$

Y así el polinomio resultante es

$$p(x) = \sum_{i=0}^n l_i(x)f(x_i)$$

Donde $f(x_i)$ es el valor de la función $f$ en $x_i$.

In [ ]:
def polinomio_lagrange(x_vals, y_vals, var=x):
    lista_L, p, n = [], S(0), len(x_vals)
    for i in range(n):
        li = S(1)
        for k in range(n):
            if k != i:
                li *= (var - x_vals[k])/(x_vals[i] - x_vals[k])
        
        lista_L.append(li)
        p += y_vals[i] * li
    
    return p, lista_L

In [ ]:
f = E**(sin(4*x))
I = [0, 1]
x_vals = np.linspace(I[0], I[1], 4)
y_vals = [f.evalf(subs={x: S(i)}) for i in x_vals]

In [ ]:
p, lista_L = polinomio_lagrange(x_vals, y_vals)

In [ ]:
expand(p)

In [ ]:
x_range = np.linspace(I[0], I[1], 100)
y_real = [f.subs(x, i) for i in x_range]
y_pol = [expand(p).subs(x, i) for i in x_range]


plt.plot(x_range, y_real, label=f'{f}')
plt.scatter(x_vals, y_vals, c="#800000")
plt.plot(x_range, y_pol, label='interpolacion', c="#bcbcbc")

plt.legend()

Ahora ploteamos cada una de las funciones

In [ ]:
x_range = np.linspace(I[0], I[1], 100)
y_pol = [expand(p).subs(x, i) for i in x_range]
y_real = [f.subs(x, i) for i in x_range]


for p_i, pl in enumerate(lista_L):
    y_l = [expand(pl).subs(x, i) * y_vals[p_i] for i in x_range]
    plt.plot(x_range, y_l, label=f'$l_{p_i}$')
    plt.scatter(x_vals[p_i], y_vals[p_i])

plt.plot(x_range, y_real, label=f'{f}')
plt.plot(x_range, y_pol, label='interpolacion', c="#bcbcbc")

plt.legend(bbox_to_anchor=(1.05, 0.5))

## Método de Newton
El método de Lagrange, aunque da un polinomio único, tiene una serie de problemas. En primer lugar, aunque computacionalmente no parezca complejo, el cálculo de coeficientes es tedioso y poco práctico para ciertas funciones. Además, si queremos añadir un nuevo nodo, tenemos que recalcular todas las funciones de Lagrange para el polinomio nuevo, lo cual lo vuelve bastante impráctico. 

Con el método de Newton solventamos las problemáticas estableciendo un método iterativo que incorpora un nodo por vez.

En el método se aplican los siguiqntes cálculos:

$$
\begin{matrix}
x_0 & \color{blue}{f(x_0)} & & \\
  & & \color{blue}{\frac{f(x_1) - f(x_0)}{x_1 - x_0}} & \\
x_1   & f(x_1) & & \color{blue}{\frac{\frac{f(x_2) - f(x_1)}{x_2 - x_1} - \frac{f(x_1) - f(x_0)}{x_1 - x_0}}{x_2 - x_0}}\\
    & & \frac{f(x_2) - f(x_1)}{x_2 - x_1}& \\
x_2     &f(x_2) & & \\
\vdots     & \vdots & & \\
x_{n-1}     &f(x_{n-1}) & & \\
& & \frac{f(x_n) - f(x_{n-1})}{x_n - x_{n-1}} & \\
x_n     &f(x_n) & & \\
\end{matrix}
$$

O de manera compacta:
$$
\begin{matrix}
x_0 & \color{blue}{f(x_0)} & & \\
  & & \color{blue}{f[x_0, x_1]} & \\
x_1   & f(x_1) & & \color{blue}{f[x_0, x_1, x_2]}\\
    & & \color{blue}{f[x_1, x_2]}& \\
x_2     &f(x_2) & & \\
\vdots     & \vdots & & \\
x_{n-1}     &f(x_{n-1}) & & \\
& & \color{blue}{f[x_{n-1}, x_n]} & \\
x_n     &f(x_n) & & \\
\end{matrix}
$$ 
Con
$$f[x_a, x_{a+1}, \cdots, x_j] = \frac{f[x_{a+1}, x_{a + 2}, \cdots, x_j] - f[x_{a}, x_{a+1}, \cdots, x_{j-1}]}{x_j - x_a}$$
Si denominamos $a_0, a_1, \cdots, a_n$ a los monomios en azul ($a_0 = f(x_0)$, $a_1 = \frac{f(x_1) - f(x_0)}{x_1 - x_0}$, etc.) entonces el polinomio de Newton se construye como:
$$p_n(x) = a_0 + a_1(x - x_0) + a_2(x - x_0)(x - x_1) + \cdots + a_n(x - x_0)(x - x_1)\cdots(x - x_{n-1})$$


In [ ]:
def polinomio_newton(x_vals, y_vals, var=x, evalf=None):
    matriz_coeffs = zeros(len(y_vals), len(y_vals))
    for i in range(len(y_vals)): # Asignamos la primera columna como f(x)
        matriz_coeffs[i,0] = y_vals[i]
    
    for col in range(1, len(y_vals)):
        for row in range(len(y_vals) - col):
            num = matriz_coeffs[row + 1, col - 1] - matriz_coeffs[row, col - 1]
            den = x_vals[row + col] - x_vals[row]
        
            matriz_coeffs[row, col] = num / den
    
    matriz_coeffs = nsimplify(matriz_coeffs,tolerance=1e-10,rational=True)  # Esto es importante para quitar valores de redondeo
    if evalf is not None: # si evalf es un numero redondea a ese numero de decimales
        matriz_coeffs = matriz_coeffs.evalf(evalf)
        
    # Aqui hacemos el polinomio
    p = matriz_coeffs[0, 0]
    for col in range(1, len(y_vals)):
        p_col = S(1)
        for i in range(0, col):
            p_col *= (var - x_vals[i])
        
        p += p_col * matriz_coeffs[0, col]
    
        
    return p, matriz_coeffs

In [ ]:
p, mat = polinomio_newton([S(i) for i in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7]], [S(i) for i in [0.7, 0.8, 1, 1.15, 1.25, 1.3]], x)
mat

In [ ]:
p

In [ ]:
f = E**(sin(4*x))
I = [0, 1]
x_vals = np.linspace(I[0], I[1], 4)
y_vals = [f.evalf(subs={x: S(i)}) for i in x_vals]

In [ ]:
p, mat = polinomio_newton(x_vals, y_vals, x, evalf=5)
p

In [ ]:
mat.evalf(4)

In [ ]:
x_range = np.linspace(I[0], I[1], 100)
y_real = [f.subs(x, i) for i in x_range]
y_pol = [expand(p).subs(x, i) for i in x_range]


plt.plot(x_range, y_real, label=f'{f}')
plt.scatter(x_vals, y_vals, c="#800000")
plt.plot(x_range, y_pol, label='interpolacion', c="#bcbcbc")

plt.legend()

## Error en la interpolación de Lagrange
Dado un polinomio de orden $n$, se tiene que el error máximo viene estimado por la expresión:
$$E(x) = f(x) - p_n(x) = \frac{(x-x_0)(x-x_1)\cdots(x-x_n)}{(n+1)!}\max_{t\in[a,b]}f^{(n+1)}(t)$$

In [ ]:
def error_lagrange(f, x_vals, I, var=x):
    E_x_fact = S(1)
    for x_val in x_vals:
        E_x_fact *= (var - x_val)
        
    E_x_fact /= factorial(len(x_vals))
    
    # Ahora derivamos la función n veces
    diff_f = f
    
    for _ in range(len(x_vals)):
        diff_f = diff_f.diff(var)
        
    max_diff_f = maximum(diff_f, var, Interval(I[0], I[1]))
    
    E_x = E_x_fact *  abs(max_diff_f)
    
    return E_x

In [ ]:
def error_maximo_estocastico(f, grado, N=1000, list_x_vals=None):
    max_error = 0

    # Hacemos la funcion por separado porque el cálculo de derivada es costoso y cada iteración de error_lagrange tarda mucho
    diff_f = f
    for _ in range(grado):
        diff_f = diff_f.diff(x)
    max_diff_f = maximum(diff_f, x, Interval(I[0], I[1]))
    
    
    for i in tqdm(range(N)):
        if list_x_vals is None:
            x_list = np.sort(np.random.rand(grado) * (I[1] - I[0]) + I[0])
        else:
            x_list = list_x_vals
            
        E_x_fact = S(1) / S(factorial(grado))
        for x_val in x_list:
            E_x_fact *= (x - x_val)

        E_x = lambdify(x, expand(E_x_fact * max_diff_f))  # lambdify es para que la evaluación numérica sea mucho más ágil

        x_range = np.linspace(I[0], I[1], int(N/10))
        y_f = [E_x(i) for i in x_range]
        max_y_f = np.max(abs(np.array(y_f)))

        if max_y_f >= max_error:
            max_error = max_y_f
        
        if list_x_vals is not None:
            break
        
    return max_error

#### EJERCICIO 48

In [ ]:
f = sqrt(x)
I = [0, 2]
grado = 4

x_vals = [1, 2]
y_vals = [f.evalf(subs={x: S(i)}) for i in x_vals]

In [ ]:
error_lagrange(f, np.linspace(I[0], I[1], grado), I)  # Asegurate de que el máximo de la derivada para el grado no es infinito!!!!

Este ejercicio lo vamos a resolver de manera aleatoria, usando una simulación. Tenemos que simular el error dados n puntos $x_0$, $x_1$, ..., $x_n$ aleatorios.

In [ ]:
max_error = error_maximo_estocastico(f, grado=grado, N=2500)

In [ ]:
print(1/max_error)

In [ ]:
x_vals = np.sort(np.random.rand(grado) * (I[1] - I[0]) + I[0])
x_vals = [0, 0.366, 1.333, 2]
y_vals = [f.evalf(subs={x: S(i)}) for i in x_vals]

p, lista_L = polinomio_lagrange(x_vals, y_vals)

x_range = np.linspace(I[0], I[1], 100)
y_real = [f.subs(x, i) for i in x_range]
y_pol = np.array([expand(p).subs(x, i) for i in x_range])


plt.plot(x_range, y_real, label=f'{f}')
plt.scatter(x_vals, y_vals, c="#800000")
plt.plot(x_range, y_pol, label='interpolacion', c="#bcbcbc")
plt.plot(x_range, list(y_pol - (max_error)),  c="#bcbcbc", alpha=0.3)
plt.plot(x_range, list(y_pol + (max_error)),  c="#bcbcbc", alpha=0.3)


# plt.fill_between(list(x_range), list(y_pol - (max_error/2)), list(y_pol + (max_error/2)))

plt.legend()

## El fenómeno de Runge y la interpolación de Chebishev
Esta información está en el texto base pero muy sucinta y mal descrita. La info para esta sección la he sacado de https://brianheinold.net/notes/An_Intuitive_Guide_to_Numerical_Methods_Heinold.pdf

El fenómeno de Runge es que cuando tomamos una función la interpolación tiende a fallar en los extremos, sobre todo si los puntos de interpolación los tomamos de manera equidistante.
Obviamente, se pueden escoger los nodos $x_0, x_1, \cdots, x_n$ de muchas maneras, pero unas y otras van a ser mejores para generar interpolaciones más veraces fuera de los extremos.


In [ ]:
f = E**(sin(4*x))
I = [0, 1]
grado = 9

x_vals_equip = np.linspace(I[0], I[1], grado)
y_vals_equip = [f.evalf(subs={x: S(i)}) for i in x_vals_equip]

x_vals_extr = [0, 0.03, 0.07, 0.15, 0.5, 0.85, 0.93, 0.97, 1]
y_vals_extr = [f.evalf(subs={x: S(i)}) for i in x_vals_extr]

In [ ]:
p_equip, lista_L = polinomio_lagrange(x_vals_equip, y_vals_equip)
p_extr, lista_L = polinomio_lagrange(x_vals_extr, y_vals_extr)

In [ ]:
expand(p_equip)

In [ ]:
expand(p_extr)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4))
x_range_base = np.linspace(I[0] - 0.05, I[1] + 0.05, 100)
x_range_todo = np.linspace(I[0] - 1, I[1] + 0.7, 100)

for x_range_idx, x_range in enumerate([x_range_base, x_range_todo]): 
    y_real = [f.subs(x, i) for i in x_range]
    y_pol_equip = [expand(p_equip).subs(x, i) for i in x_range]
    y_pol_extr = [expand(p_extr).subs(x, i) for i in x_range]


    axs[x_range_idx].plot(x_range, y_real, label=f'{f}')
    axs[x_range_idx].scatter(x_vals_equip, y_vals_equip, c="#bcbcbc")
    axs[x_range_idx].scatter(x_vals_extr, y_vals_extr, c="#454545")

    axs[x_range_idx].plot(x_range, y_pol_equip, label='interpolacion equiprobable', c="#bcbcbc")
    axs[x_range_idx].plot(x_range, y_pol_extr, label='interpolacion extremos', c="#454545")


    axs[x_range_idx].legend()

Vemos que aunque la interpolación equidistante sea más exacta con respecto a la función real, se aleja un montón de los extremos. Sin embargo, si tomamos puntos en el extremo, la interpolación aguanta mejor cambios en el extremo.

Por el teorema del error máximo mencionado anteriormente se deriva que la major elección de valores de $x$ son las raíces de los polinomios de Chevisheb, que se corresponden a la ecuación:
$$x_k = \frac{1}{2}(a+b) + \frac{1}{2}(b - a) \cos\left( \frac{2k - 1}{2n} \pi \right)$$
Para $k = 1, \cdots, n$

In [ ]:
def roots_chebyshev(n, var=x, I=[-1, 1]):
    roots = [0.5*(I[1] + I[0]) + 0.5*(I[1] - I[0]) * cos(S(2 * (i + 1) - 1)/S(2*n) * pi) for i in range(n)][::-1]
    return roots

In [ ]:
for n in range(2, 20):
    plt.scatter(roots_chebyshev(n, I=[0, 1]), [n] * n)

Repetimos el ejemplo anterior, pero ahora tomando las raíces de Chebyshev

In [ ]:
x_vals_chev = [N(i, 6) for i in roots_chebyshev(grado, I=I)]
y_vals_chev = [f.evalf(subs={x: S(i)}) for i in x_vals_chev]

In [ ]:
p_chev, lista_L = polinomio_lagrange(x_vals_chev, y_vals_chev)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 4))
x_range_base = np.linspace(I[0] - 0.05, I[1] + 0.05, 100)
x_range_todo = np.linspace(I[0] - 1, I[1] + 0.7, 100)

for x_range_idx, x_range in enumerate([x_range_base, x_range_todo]): 
    y_real = [f.subs(x, i) for i in x_range]
    y_pol_equip = [expand(p_equip).subs(x, i) for i in x_range]
    y_pol_chev = [expand(p_chev).subs(x, i) for i in x_range]
    y_pol_extr = [expand(p_extr).subs(x, i) for i in x_range]


    axs[x_range_idx].plot(x_range, y_real, label=f'{f}')
    axs[x_range_idx].scatter(x_vals_equip, y_vals_equip, c="#bcbcbc")
    axs[x_range_idx].scatter(x_vals_extr, y_vals_extr, c="#454545")
    axs[x_range_idx].scatter(x_vals_chev, y_vals_chev, c="#800000")


    axs[x_range_idx].plot(x_range, y_pol_equip, label='equiprobable', c="#bcbcbc")
    axs[x_range_idx].plot(x_range, y_pol_extr, label='extremos', c="#454545")
    axs[x_range_idx].plot(x_range, y_pol_chev, label='chebyshev', c="#800000")


    axs[x_range_idx].legend()

Aunque los resultados en los extremos sean peor para Chebyshev que para la interpolación de extremos, se ve que en el medio la interpolación de Chebyshev es prácticamente idéntica a la equiprobable. Esto es importante porque para esta función las diferencias en el centro para la interpolación de extremos no son muy evidentes, pero para otras funciones puede que sean peores, y en ese caso la selección de nodos por Chebyshev es ese punto intermedio.